In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import date, time, datetime

In [2]:
train_users = pd.read_csv("proj3_data/train_users.csv")
test_users = pd.read_csv("proj3_data/test_users.csv")
sessions = pd.read_csv("proj3_data/sessions.csv")


In [3]:
Y = train_users['country_destination']
train_users = train_users.drop('country_destination', axis =1)

In [4]:
train_users.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome


In [5]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [6]:
#train_users.drop(['country_destination'], axis = 1)
#Concatenating train and test data for EDA
df_all = pd.concat((train_users, test_users), axis = 0, ignore_index= True)

In [7]:
dates_list = []
year_list = []
month_list = []

for i in df_all["date_account_created"]:
    date_time_obj = datetime.strptime(i, '%Y-%m-%d')
    year = date_time_obj.date().year
    month = date_time_obj.date().month
    dates_list.append(date_time_obj)
    year_list.append(year)
    month_list.append(month)

df_all["datetime_account_created"] = dates_list
df_all["year_account_created"] = year_list
df_all["month_account_created"] = month_list

In [8]:
dates_list = []
year_list = []
month_list = []

for i in df_all["timestamp_first_active"]:
    time_stamp_obj = pd.to_datetime(i , format='%Y%m%d%H%M%S')
    year = time_stamp_obj.date().year
    month = time_stamp_obj.date().month
    dates_list.append(time_stamp_obj)
    year_list.append(year)
    month_list.append(month)
    
df_all["datetime_first_active"] = dates_list
df_all["year_first_active"] = year_list
df_all["month_first_active"] = month_list

In [9]:
df_all["created_after_active"] = df_all["datetime_account_created"] - df_all["datetime_first_active"]

In [10]:
df_all['age'].quantile(0.5)

33.0

In [11]:
df_all['age'] = np.where(df_all['age'] > 110, 34, df_all['age'])
df_all['age'].describe()

count    158681.000000
mean         36.838115
std          13.766690
min           1.000000
25%          28.000000
50%          33.000000
75%          42.000000
max         110.000000
Name: age, dtype: float64

In [12]:
by_signup_device = df_all.groupby(['signup_method', 'first_device_type'])

In [13]:
def impute_median(series):
    return series.fillna(series.median())

In [14]:
df_all.age = by_signup_device['age'].transform(impute_median)

/Users/amangangwani/miniconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [15]:
df_all['first_affiliate_tracked'].value_counts()

untracked        143181
linked            62064
omg               54859
tracked-other      6655
product            2353
marketing           281
local ops            69
Name: first_affiliate_tracked, dtype: int64

In [16]:
tracked = []

for i in df_all['first_affiliate_tracked']:
    if i == "untracked" or i == "":
        isTracked = 0
    else:
        isTracked = 1
    tracked.append(isTracked)

df_all['is_first_affiliate_tracked'] = tracked

In [17]:
df_all.isnull().sum()

id                                 0
date_account_created               0
timestamp_first_active             0
date_first_booking            186639
gender                             0
age                                1
signup_method                      0
signup_flow                        0
language                           0
affiliate_channel                  0
affiliate_provider                 0
first_affiliate_tracked         6085
signup_app                         0
first_device_type                  0
first_browser                      0
datetime_account_created           0
year_account_created               0
month_account_created              0
datetime_first_active              0
year_first_active                  0
month_first_active                 0
created_after_active               0
is_first_affiliate_tracked         0
dtype: int64

In [18]:
df_all['created_after_active'].describe()

count                         275547
mean     -1 days +15:36:42.825191347
std       10 days 12:30:22.643182702
min                -1 days +00:00:01
25%                -1 days +03:53:56
50%                -1 days +08:24:51
75%                -1 days +19:22:57
max               1455 days 09:53:18
Name: created_after_active, dtype: object

In [19]:
df_all = df_all.rename(columns={"id":"user_id"})

In [20]:
sessions.user_id.value_counts()

mxqbh3ykxl    2722
0hjoc5q8nf    2644
mjbl6rrj52    2476
l5lgm3w5pc    2424
wg9413iaux    2362
              ... 
gir5jraxkf       1
0rjz5z5m7s       1
clmnizbi6q       1
qidcm4f69a       1
5c2eh6f2fp       1
Name: user_id, Length: 135483, dtype: int64

In [21]:
seconds = sessions.groupby('user_id', as_index=False).agg({"secs_elapsed": "sum"})
seconds.head()

,user_id,secs_elapsed
0,00023iyk9l,867896.0
1,0010k6l0om,586543.0
2,001wyh0pz8,282965.0
3,0028jgx1x1,297010.0
4,002qnbzfs5,6487080.0


In [22]:
df_all = pd.merge(df_all, seconds, on="user_id", how="left")

In [23]:
df_all.head()

,user_id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_browser,datetime_account_created,year_account_created,month_account_created,datetime_first_active,year_first_active,month_first_active,created_after_active,is_first_affiliate_tracked,secs_elapsed
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,33.0,facebook,0,en,direct,...,Chrome,2010-06-28,2010,6,2009-03-19 04:32:55,2009,3,465 days 19:27:05,0,NaN
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,...,Chrome,2011-05-25,2011,5,2009-05-23 17:48:09,2009,5,731 days 06:11:51,0,NaN
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,...,IE,2010-09-28,2010,9,2009-06-09 23:12:47,2009,6,475 days 00:47:13,0,NaN
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,...,Firefox,2011-12-05,2011,12,2009-10-31 06:01:29,2009,10,764 days 17:58:31,0,NaN
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,...,Chrome,2010-09-14,2010,9,2009-12-08 06:11:05,2009,12,279 days 17:48:55,0,NaN


In [24]:
df_all['secs_elapsed'] = df_all['secs_elapsed'].fillna(0)

In [25]:
df_all.head()

,user_id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_browser,datetime_account_created,year_account_created,month_account_created,datetime_first_active,year_first_active,month_first_active,created_after_active,is_first_affiliate_tracked,secs_elapsed
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,33.0,facebook,0,en,direct,...,Chrome,2010-06-28,2010,6,2009-03-19 04:32:55,2009,3,465 days 19:27:05,0,0.0
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,...,Chrome,2011-05-25,2011,5,2009-05-23 17:48:09,2009,5,731 days 06:11:51,0,0.0
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,...,IE,2010-09-28,2010,9,2009-06-09 23:12:47,2009,6,475 days 00:47:13,0,0.0
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,...,Firefox,2011-12-05,2011,12,2009-10-31 06:01:29,2009,10,764 days 17:58:31,0,0.0
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,...,Chrome,2010-09-14,2010,9,2009-12-08 06:11:05,2009,12,279 days 17:48:55,0,0.0


In [26]:
#df_all['datetime_account_created'].dayofweek()

In [27]:
df_all['is_first_affiliate_tracked'].isna().sum()

0

In [28]:
df_all.gender.unique()

array(['-unknown-', 'MALE', 'FEMALE', 'OTHER'], dtype=object)

In [29]:
df_all

,user_id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_browser,datetime_account_created,year_account_created,month_account_created,datetime_first_active,year_first_active,month_first_active,created_after_active,is_first_affiliate_tracked,secs_elapsed
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,33.0,facebook,0,en,direct,...,Chrome,2010-06-28,2010,6,2009-03-19 04:32:55,2009,3,465 days 19:27:05,0,0.0
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,...,Chrome,2011-05-25,2011,5,2009-05-23 17:48:09,2009,5,731 days 06:11:51,0,0.0
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,...,IE,2010-09-28,2010,9,2009-06-09 23:12:47,2009,6,475 days 00:47:13,0,0.0
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,...,Firefox,2011-12-05,2011,12,2009-10-31 06:01:29,2009,10,764 days 17:58:31,0,0.0
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,...,Chrome,2010-09-14,2010,9,2009-12-08 06:11:05,2009,12,279 days 17:48:55,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275542,cv0na2lf5a,2014-09-30,20140930235232,NaN,-unknown-,31.0,basic,0,en,direct,...,IE,2014-09-30,2014,9,2014-09-30 23:52:32,2014,9,-1 days +00:07:28,0,2534232.0
275543,zp8xfonng8,2014-09-30,20140930235306,NaN,-unknown-,32.0,basic,23,ko,direct,...,-unknown-,2014-09-30,2014,9,2014-09-30 23:53:06,2014,9,-1 days +00:06:54,0,51618.0
275544,fa6260ziny,2014-09-30,20140930235408,NaN,-unknown-,34.0,basic,0,de,direct,...,Firefox,2014-09-30,2014,9,2014-09-30 23:54:08,2014,9,-1 days +00:05:52,1,823297.0
275545,87k0fy4ugm,2014-09-30,20140930235430,NaN,-unknown-,34.0,basic,0,en,sem-brand,...,Safari,2014-09-30,2014,9,2014-09-30 23:54:30,2014,9,-1 days +00:05:30,1,352068.0


In [30]:
list(df_all.columns)

['user_id',
 'date_account_created',
 'timestamp_first_active',
 'date_first_booking',
 'gender',
 'age',
 'signup_method',
 'signup_flow',
 'language',
 'affiliate_channel',
 'affiliate_provider',
 'first_affiliate_tracked',
 'signup_app',
 'first_device_type',
 'first_browser',
 'datetime_account_created',
 'year_account_created',
 'month_account_created',
 'datetime_first_active',
 'year_first_active',
 'month_first_active',
 'created_after_active',
 'is_first_affiliate_tracked',
 'secs_elapsed']

In [31]:
#columns = []

In [32]:
df_all['day_account_created'] = df_all['datetime_account_created'].dt.day_name()
df_all['day_first_active'] = df_all['datetime_first_active'].dt.day_name()

In [33]:
day_acct_created = pd.get_dummies(df_all['day_account_created'])
day_first_active = pd.get_dummies(df_all['day_first_active'])

In [34]:
day_acct_created = day_acct_created.rename(columns={"Monday": "acct_created_Mon", "Tuesday": "acct_created_Tues", "Wednesday": "acct_created_Wed",
                                "Thursday": "acct_created_Thurs", "Friday": "acct_created_Fri", "Saturday": "acct_created_Sat", 
                                 "Sunday": "acct_created_Sun"})
day_first_active = day_first_active.rename(columns={"Monday": "first_active_Mon", "Tuesday": "first_active_Tues", "Wednesday": "first_active_Wed",
                                "Thursday": "first_active_Thurs", "Friday": "first_active_Fri", "Saturday": "first_active_Sat", 
                                 "Sunday": "first_active_Sun"})

In [35]:
df_all = pd.concat([df_all.reset_index(drop=True),day_acct_created.reset_index(drop=True)], axis=1)
df_all = pd.concat([df_all.reset_index(drop=True),day_first_active.reset_index(drop=True)], axis=1)

In [36]:
method = pd.get_dummies(df_all[["signup_method"]])
affch = pd.get_dummies(df_all[["affiliate_channel"]])
affprov = pd.get_dummies(df_all[["affiliate_provider"]])
firstdevice = pd.get_dummies(df_all[["first_device_type"]])
browser = pd.get_dummies(df_all[["first_browser"]])
signup = pd.get_dummies(df_all[["signup_app"]])
genderdum = pd.get_dummies(df_all[["gender"]])
langdum = pd.get_dummies(df_all[["language"]])




In [37]:
df2 = df_all

In [38]:
df2 = pd.concat([df2.reset_index(drop=True),method.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),affch.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),affprov.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),firstdevice.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),browser.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),genderdum.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),signup.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),langdum.reset_index(drop=True)], axis=1)

In [39]:
missing_values_table(df2)


Your selected dataframe has 168 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
date_first_booking,186639,67.7
first_affiliate_tracked,6085,2.2
age,1,0.0


In [40]:
list(df2.columns)


['user_id',
 'date_account_created',
 'timestamp_first_active',
 'date_first_booking',
 'gender',
 'age',
 'signup_method',
 'signup_flow',
 'language',
 'affiliate_channel',
 'affiliate_provider',
 'first_affiliate_tracked',
 'signup_app',
 'first_device_type',
 'first_browser',
 'datetime_account_created',
 'year_account_created',
 'month_account_created',
 'datetime_first_active',
 'year_first_active',
 'month_first_active',
 'created_after_active',
 'is_first_affiliate_tracked',
 'secs_elapsed',
 'day_account_created',
 'day_first_active',
 'acct_created_Fri',
 'acct_created_Mon',
 'acct_created_Sat',
 'acct_created_Sun',
 'acct_created_Thurs',
 'acct_created_Tues',
 'acct_created_Wed',
 'first_active_Fri',
 'first_active_Mon',
 'first_active_Sat',
 'first_active_Sun',
 'first_active_Thurs',
 'first_active_Tues',
 'first_active_Wed',
 'signup_method_basic',
 'signup_method_facebook',
 'signup_method_google',
 'signup_method_weibo',
 'affiliate_channel_api',
 'affiliate_channel_cont

In [44]:
#columns_to_drop = ['user_id', 'date_account_created', 'timestamp_first_active', 'date_first_booking',
#'gender', 'signup_method', 'signup_flow',affiliate_channel, affiliate_provider, 
#first_device_type, first_browser, signup_app
df2 = df2.drop(['user_id', 'date_account_created', 'timestamp_first_active', 'date_first_booking','gender', 
          'signup_method', 'signup_flow', 'affiliate_channel', 'affiliate_provider', 'first_device_type', 
          'first_browser', 'signup_app'], axis=1)


KeyError: "['user_id' 'date_account_created' 'timestamp_first_active'\n 'date_first_booking' 'gender' 'signup_method' 'signup_flow'\n 'affiliate_channel' 'affiliate_provider' 'first_device_type'\n 'first_browser' 'signup_app'] not found in axis"

In [45]:
df2.head(5)
df2 = df2.drop('language', axis=1)

In [48]:
df2 = df2.drop(['first_affiliate_tracked', 'datetime_first_active', 'datetime_account_created'] ,axis=1)

In [50]:
df2 = df2.drop(['created_after_active', 'day_account_created', 'day_first_active'] ,axis=1)

In [51]:
df2.head(5)

,age,year_account_created,month_account_created,year_first_active,month_first_active,is_first_affiliate_tracked,secs_elapsed,acct_created_Fri,acct_created_Mon,acct_created_Sat,...,language_ko,language_nl,language_no,language_pl,language_pt,language_ru,language_sv,language_th,language_tr,language_zh
0,33.0,2010,6,2009,3,0,0.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,38.0,2011,5,2009,5,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,56.0,2010,9,2009,6,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42.0,2011,12,2009,10,0,0.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,41.0,2010,9,2009,12,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
from sklearn import model_selection

train = df2.iloc[0:len(train_users),:]
test = df2.iloc[len(train_users):,:]


X_train, X_test, y_train, y_test = model_selection.train_test_split(train, Y, test_size=0.33, random_state=42)

In [85]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62096 entries, 213451 to 275546
Columns: 149 entries, age to language_zh
dtypes: float64(2), int64(5), uint8(142)
memory usage: 11.7 MB


In [54]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  

In [61]:
model = xgb.fit(X_train,y_train)

In [57]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=25, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=0, subsample=0.5,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [65]:
from sklearn.metrics import ndcg_score

model.score(X_train, y_train)

0.6330307946186334

In [67]:
y_pred = model.predict(X_test)

In [69]:
#ndcg_score(y_test, y_pred)

In [86]:
Y_PREDICT = model.predict(test)

In [90]:
submission = pd.DataFrame(Y_PREDICT)
test_users = pd.read_csv("proj3_data/test_users.csv")
submission["id"] = test_users["id"]
submission.columns = ['country', 'id']
submission = submission[['id', 'country']]
submission

,id,country
0,5uwns89zht,NDF
1,jtl0dijy2j,NDF
2,xx0ulgorjt,NDF
3,6c6puo6ix0,NDF
4,czqhjk3yfe,NDF
...,...,...
62091,cv0na2lf5a,NDF
62092,zp8xfonng8,NDF
62093,fa6260ziny,NDF
62094,87k0fy4ugm,NDF


In [91]:
submission.to_csv("submission.csv")
